# 파인튜닝
- 출력층 등을 변경한 모델을 학습된 모델을 기반으로 구축한 후 직접 준비한 데이터로 신경망 모델의 결합 파라미터를 학습 시키는 방법
- 결합 파라미터의 초기값은 학습된 모델의 파라미터를 사용
- 일반적으로 입력층에 가까운 부분의 파라미터는 학습률을 작게(경우에 따라서 변화 없이), 출력층에 가까운 부분의 파라미터는 학습률을 크게 설정
- 전이학습처럼 학습된 모델을 기반으로 하는 파인튜닝은 직접 준비한 데이터가 적어도 높은 성능의 딥러닝을 실현하기 쉬움

In [1]:
!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

fatal: destination path 'pytorch_advanced' already exists and is not an empty directory.


In [2]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [3]:
%cd 1_image_classification

/content/pytorch_advanced/1_image_classification


In [4]:
# make_folders_and_data_downloads.ipynbの中身を実行
import os
import urllib.request
import zipfile


data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir, "imagenet_class_index.json")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [5]:
# 패키지 import
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [6]:
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

In [7]:
# 개미와 벌의 화상 파일 경로 리스트 작성
train_list = make_datapath_list(phase = "train")
val_list = make_datapath_list(phase = "val")

# Dataset 작성
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list = train_list, transform = ImageTransform(size, mean, std), phase = 'train'
)
val_dataset = HymenopteraDataset(
    file_list = val_list, transform = ImageTransform(size, mean, std), phase = 'val'
)

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [8]:
# DataLoader 작성
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size = batch_size, shuffle = False
)

# 사전 객체에 정리
dataloaders_dict = {"train" : train_dataloader, "val" : val_dataloader}

# 네트워크 모델 작성

In [9]:
# 학습된 VGG-16 모델 코드

# VGG-16 모델의 인스턴스 생성
use_pretrained = True # 학습된 파라미터 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력층의 출력 유닛을 개미와 벌 두 개로 변경
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
net.train()

print('네트워크 설정 완료 : 학습된 가중치를 로드하고 훈련 모드로 설정했습니다.')

네트워크 설정 완료 : 학습된 가중치를 로드하고 훈련 모드로 설정했습니다.


In [10]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 손실함수 정의 및 최적화 방법 설정

In [11]:
# 손실함수 설정
criterion = nn.CrossEntropyLoss()

파인튜닝은 최적화 기법의 설정이 전이학습과 다름

모든 층의 파라미터를 학습할 수 있도록 옵티마이저 설정

VGG-16의 전반부 feature모듈의 파라미터 -> update_param_names_1

후반부 전결합 층의 classifier 모듈 중 처음 두 개의 전결합 층 파라미터 -> update_param_names2 

교체한 마지막 전결합 층 변수 -> update_param_names_3


In [12]:
# 파인튜닝으로 학습할 파라미터를 params_to_update 변수의 1~3에 저장
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight",
                        "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각 리스트에 저장
for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print("params_to_update_1에 저장: ", name)

  elif name in update_param_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    print("params_to_update_2에 저장: ", name)
  
  elif name in update_param_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    print("params_to_update_3에 저장: ", name)
  else:
    param.requires_grad = False
    print("경사 계산 없음. 학습하지 않음: ", name)

params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [13]:
# 모멘턴 SGD 사용
# update_param_names_1 : 1e-4
# update_param_names_2 : 5e-4
# update_param_names_3 : 1e-3
# momentum = 0.9

optimizer = optim.SGD([
                       {'params': params_to_update_1, 'lr': 1e-4},
                       {'params': params_to_update_2, 'lr': 5e-4},
                       {'params': params_to_update_3, 'lr': 1e-3}],
                      momentum = 0.9)

In [14]:
# 학습 및 검증 실시

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

  # 초기 설정
  # GPU 사용가능한지 확인
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("사용장치: ", device)

  # 네트워크를 GPU로
  net.to(device)

  # 네트워크가 어느 정도 고정되면 고속화시킨다.
  torch.backends.cudnn.benchmark = True

  # 에폭 루프
  for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch+1, num_epochs))
      print('----------')

      # 에폭별 훈련 및 검증 루프
      for phase in ['train', 'val']:
        if phase == 'train':
          net.train() # 모델을 훈련 모드로
        else:
          net.eval() # 모델을 검증 모드로

        epoch_loss = 0.0 # 에폭 손실 합
        epoch_corrects = 0 # 에폭 정답 수

        # 학습하지 않았을 때의 검증 성능을 확인하기 위해 epoch=0의 훈련 생략
        if (epoch == 0) and (phase == 'train'):
          continue
      
        # 데이터 로더에서 미니 배치를 꺼내 루프
        for inputs, labels in tqdm(dataloaders_dict[phase]):

          # GPU를 사용할 수 있다면 GPU에 데이터르 ㄹ보낸다.
          inputs = inputs.to(device)
          labels = labels.to(device)

          # 옵티마이저 초기화
          optimizer.zero_grad()

          # 순전파 계산
          with torch.set_grad_enabled(phase == 'train'):
            outputs = net(inputs)
            loss = criterion(outputs, labels) # 손실 계산
            _, preds = torch.max(outputs, 1)

            # 훈련 시에는 오차 역전파법
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # 결과 계산
            epoch_loss += loss.item() * inputs.size(0) # 손실의 합계 갱신
            # 정답 수의 합계 갱신
            epoch_corrects += torch.sum(preds == labels.data)

        # 에폭별 손실과 정답률 표시
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

Parameter containing:
tensor([[[[-0.5535,  0.1429,  0.5291],
          [-0.5830,  0.3567,  0.7658],
          [-0.6901, -0.0479,  0.4842]],

         [[ 0.1757,  0.0100, -0.0813],
          [ 0.0443, -0.0702, -0.2603],
          [ 0.1326, -0.1727, -0.1322]],

         [[ 0.3133, -0.1657, -0.4273],
          [ 0.4755, -0.0824, -0.4868],
          [ 0.6323,  0.0196, -0.2773]]],


        [[[ 0.2332,  0.1273,  0.1867],
          [-0.4274, -0.2429,  0.2469],
          [-0.2501,  0.1423, -0.0050]],

         [[-0.1404, -0.2187,  0.1507],
          [-0.8410, -0.3515,  0.5642],
          [-0.2417,  0.5195,  0.5393]],

         [[-0.3141, -0.3703, -0.1308],
          [-0.4713, -0.1549,  0.3460],
          [ 0.0545,  0.5869,  0.4959]]],


        [[[ 0.1773,  0.5216,  0.0101],
          [-0.2717, -0.7169,  0.3131],
          [-0.0755, -0.2206,  0.3348]],

         [[ 0.3094,  0.6709,  0.0208],
          [-0.4658, -1.0695,  0.3353],
          [-0.0800, -0.3049,  0.5449]],

         [[ 0.3156,  0

In [15]:
# 학습 및 검증 실행
num_epochs = 50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


사용장치:  cuda:0
Epoch 1/50
----------


100%|██████████| 5/5 [00:02<00:00,  1.68it/s]


val Loss: 0.7704 Acc: 0.4444
Epoch 2/50
----------


100%|██████████| 8/8 [00:06<00:00,  1.26it/s]


train Loss: 0.5094 Acc: 0.7078


...

val Loss: 0.1574 Acc: 0.9477
Epoch 50/50
----------


100%|██████████| 8/8 [00:03<00:00,  2.34it/s]


train Loss: 0.0018 Acc: 1.0000


100%|██████████| 5/5 [00:01<00:00,  3.28it/s]

val Loss: 0.1610 Acc: 0.9542


In [23]:
for p in net.parameters(): 
  print(p)

Parameter containing:
tensor([[[[-0.5535,  0.1429,  0.5291],
          [-0.5830,  0.3567,  0.7658],
          [-0.6901, -0.0479,  0.4842]],

         [[ 0.1757,  0.0100, -0.0813],
          [ 0.0443, -0.0702, -0.2603],
          [ 0.1326, -0.1727, -0.1322]],

         [[ 0.3133, -0.1657, -0.4273],
          [ 0.4755, -0.0824, -0.4868],
          [ 0.6323,  0.0196, -0.2773]]],


        [[[ 0.2332,  0.1273,  0.1867],
          [-0.4274, -0.2429,  0.2469],
          [-0.2501,  0.1423, -0.0050]],

         [[-0.1404, -0.2187,  0.1507],
          [-0.8410, -0.3515,  0.5642],
          [-0.2417,  0.5195,  0.5393]],

         [[-0.3141, -0.3703, -0.1308],
          [-0.4713, -0.1549,  0.3460],
          [ 0.0545,  0.5869,  0.4959]]],


        [[[ 0.1773,  0.5216,  0.0101],
          [-0.2717, -0.7169,  0.3131],
          [-0.0755, -0.2206,  0.3348]],

         [[ 0.3094,  0.6709,  0.0208],
          [-0.4658, -1.0695,  0.3353],
          [-0.0800, -0.3049,  0.5449]],

         [[ 0.3156,  0

In [24]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 학습한 네트워크 저장 및 코드
net 변수를 .state_dict()를 활용해 파라미터를 사전 변수로 꺼낸 후 torch.save()로 저장

In [17]:
# 파이토치 네트워크 파라미터 저장
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [18]:
# 파이토치 네트워크 파라미터 로드
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU 상에 저장된 가중치를 CPU에 로드할 경우
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>

In [22]:
load_weights

OrderedDict([('features.0.weight', tensor([[[[-0.5535,  0.1429,  0.5291],
                        [-0.5830,  0.3567,  0.7658],
                        [-0.6901, -0.0479,  0.4842]],
              
                       [[ 0.1757,  0.0100, -0.0813],
                        [ 0.0443, -0.0702, -0.2603],
                        [ 0.1326, -0.1727, -0.1322]],
              
                       [[ 0.3133, -0.1657, -0.4273],
                        [ 0.4755, -0.0824, -0.4868],
                        [ 0.6323,  0.0196, -0.2773]]],
              
              
                      [[[ 0.2332,  0.1273,  0.1867],
                        [-0.4274, -0.2429,  0.2469],
                        [-0.2501,  0.1423, -0.0050]],
              
                       [[-0.1404, -0.2187,  0.1507],
                        [-0.8410, -0.3515,  0.5642],
                        [-0.2417,  0.5195,  0.5393]],
              
                       [[-0.3141, -0.3703, -0.1308],
                        [-0.4713, -